## Auto mode (single uav)

This is for testing propouses

In [1]:
# Custom modules
from config import Color
from helpers import clean  #, local2global
from helpers.change_coordinates import ENUs_to_GRAs, rel_to_abs
from mavlink.customtypes.location import ENU, ENUPose, GRAPose
from mavlink.util import save_mission
from plan import Plan
from simulators import QGC, ConfigGazebo, Gazebo, NoneVisualizer, Simulator

clean()

## Create mission

In [2]:
mission_name = "square"
offset = ENUPose(0,0,0,0)  # east, north, up, heading 
origin = GRAPose(-35.363261, 149.165230, 0, 0)
rel_path = Plan.create_square_path(side_len=10, alt=5)

## Convert to Global (ENU) coordinates

In [3]:
home = ENU(*offset[:3])
abs_path = [rel_to_abs(pos, home, cls=ENU) for pos in rel_path]

## Create Plan

In [4]:
gra_offset = ENUs_to_GRAs(origin, [offset])[0]
gra_path = ENUs_to_GRAs(origin, abs_path)
save_mission(name=mission_name,poses=gra_path)

In [5]:
gra_path 

[GRAPose(lat=-35.363261, lon=149.16523, alt=5, heading=0.0),
 GRAPose(lat=-35.36317116795047, lon=149.16523, alt=5, heading=0.0),
 GRAPose(lat=-35.36317116790048, lon=149.16534015579612, alt=5, heading=0.0),
 GRAPose(lat=-35.36326099995002, lon=149.16534015591867, alt=5, heading=0.0),
 GRAPose(lat=-35.363261, lon=149.16523, alt=5, heading=0.0)]

## Visualization Parameters

In [ ]:
markers = [
	ConfigGazebo.create_markertraj(traj=abs_path, color=Color.BLUE)
]
Gazebo.plot_3d_interactive(
	markers=markers,
	title="Simulation Waypoints",
	frames=(0.2, 0.2, 0.6),
	ground=-0.05
)

## Visualizer Config

In [7]:
no_vis=NoneVisualizer(origin,[offset])


gazebo_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway.world",
                           models=[("iris",color)],
                           marker_trajs = markers)
gaz_vis= Gazebo(origin,[offset],gazebo_config)

qgc_vis=QGC(origin,[offset])

## Launch Simulatotion

In [ ]:
# TODO: Is it possible/feasible use more than one visualizer at the same time? 
# It seems there is a port conflict, only the visualizer that launches first works.

simulator = Simulator(
	visualizers=[gaz_vis],
	terminals=False,
	verbose=2
)
orac = simulator.launch(gcs_sysids={'blue 🟦': [1]})

🗺️ QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.
🚀 ArduPilot SITL vehicle 1 launched (PID 3098084)
🚀 UAV logic for vehicle 1 launched (PID 3098085)
🚀 Proxy for vehicle 1 launched (PID 3098086)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
System id: 1
[Errno 111] Connection refused sleeping
System id: 1
SIM_VEHICLE: Start
SIM_VEHICLE: Killing tasks
SIM_VEHICLE: Starting up at [-35.363261, 149.16523, 0.0, 0.0]
SIM_VEHICLE: Using defaults from (../../../ardupilot/Tools/autotest/default_params/copter.parm)
SIM_VEHICLE: Adding parameters from (/home/abeldg/repo_temp/uav-cyber-sim/params/vehicle.parm)
Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=1
Suggested EK3_DRAG_BCOEF_* = 17.209, EK3_DRAG_MCOEF = 0.209
Home: -35.363261 149.165230 alt=0.000000m hdg=0.000000
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9005
Waiting for connection ....


bind port 5760 for SERIAL0
SERIAL0 on TCP port 5760


Connection on serial port 5760

🚀 Starting Proxy 1
Loaded defaults from ../../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/repo_temp/uav-cyber-sim/params/vehicle.parm
Smoothing reset at 0.001
Loaded defaults from ../../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/repo_temp/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/repo_temp/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/repo_temp/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/repo_temp/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/repo_temp/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../../ardupilot/Tools/autotest/default_params/copter.parm,/home/ab

bind port 5762 for SERIAL1
SERIAL1 on TCP port 5762
bind port 5763 for SERIAL2
SERIAL2 on TCP port 5763
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)


🔗 UAV logic 1 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 1 logic
Vehicle 1: ▶️ Plan Started: 📋 square_auto
🚀 GCS blue 🟦 launched (PID 3098258)


## Execute Plan

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 1: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 1: ▶️ Step Started: clear uav missions
🔗 UAV logic 1 is connected
SIM_VEHICLE: Waiting for SITL to exit
🧹 Cleared previous mission
Vehicle 1: ✅ Step Done: clear uav missions
✅ 7 waypoints read.
🧭 Mission[0] → cmd: WAYPOINT, x: -35.363261, y: 149.16523, z: 0.0, current: 0
🧭 Mission[1] → cmd: TAKEOFF, x: -35.363261, y: 149.16523, z: 5.0, current: 0
🧭 Mission[2] → cmd: WAYPOINT, x: -35.3631712, y: 149.16523, z: 5.0, current: 0
🧭 Mission[3] → cmd: WAYPOINT, x: -35.3631712, y: 149.1653402, z: 5.0, current: 0
🧭 Mission[4] → cmd: WAYPOINT, x: -35.363261, y: 149.1653402, z: 5.0, current: 0
🧭 Mission[5] → cmd: WAYPOINT, x: -35.363261, y: 149.16523, z: 5.0, current: 0
🧭 Mission[6] → cmd: LAND, x: -35.363261, y: 149.16523, z: 0.0, current: 0
New connection on SERIAL1
✅ Sent mission item 0
✅ Sent mission item 1
✅ Sent mission item 2
✅ Sent mission item 3
✅ Sent mission item 4
✅ Sent mission item 5
✅ Sent mission item 6
Vehicle 1: ▶️ Step Start

📤 GCS ← UAV 1: Sending DONE (attempt 2)
📤 GCS ← UAV 1: Sending DONE (attempt 3)
📤 GCS ← UAV 1: Sending DONE (attempt 4)
📤 GCS ← UAV 1: Sending DONE (attempt 5)
📤 GCS ← UAV 1: Sending DONE (attempt 6)
📤 GCS ← UAV 1: Sending DONE (attempt 7)
📤 GCS ← UAV 1: Sending DONE (attempt 8)
📤 GCS ← UAV 1: Sending DONE (attempt 9)
📤 GCS ← UAV 1: Sending DONE (attempt 10)
✅ Vehicle 1 completed its mission
✅ All UAVs assigned to GCS blue 🟦 have completed their mission.
💾 Trajectories saved to 'trajectories_blue 🟦.pkl'.
✅ ACK received. DONE message acknowledged.
❎ Vehicle 1 logic stopped.
Connection reset or closed by peer on TCP socket
❎ Proxy 1 stopped.
Closed connection on SERIAL0
